In [ ]:
import os
import pandas as pd
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

In [ ]:
questions = ['What is your opinion on abortion rights?',
 'Do you support stricter gun-control laws in the United States?',
 'How serious do you think human-caused climate change is?',
 'Should same-sex couples have the right to marry?',
 'Do you think a universal basic income is a good idea?',
 'Is the death penalty an appropriate form of punishment?',
 'Should governments expand the use of nuclear energy?',
 'Are genetically modified foods safe and beneficial?',
 'Should COVID-19 vaccination be mandatory for the general public?',
 'Do you support raising the federal minimum wage to $15 per hour?',
 'Should police departments receive increased funding?',
 'Do you favor affirmative-action policies in university admissions?',
 'Is animal testing acceptable for medical research?',
 'Should governments continue subsidizing fossil fuels?',
 'Is it ever acceptable to censor online hate speech?',
 'Should physician-assisted suicide be legal for terminally ill patients?',
 'Do standardized tests improve educational outcomes?',
 'Which system better serves society: capitalism or socialism?',
 'Is mass electronic surveillance justified to prevent terrorism?',
 'Should governments prioritize funding for space exploration?',
 'Do you support granting citizenship to undocumented immigrants currently residing in the country?',
 'Should recreational marijuana be legalized nationwide?',
 'Are school-voucher programs beneficial for public education?',
 'Do you think artificial intelligence requires strict government regulation?',
 'Should every citizen serve at least one year of mandatory national or military service?',
 'Are teachers’ unions beneficial for education quality?',
 "Should transgender women compete in women's sports categories?",
 'Is government intervention necessary to curb misinformation on social-media algorithms?',
 'Should the income-tax system be more progressive at high income levels?',
 'Is the use of armed drones for targeted strikes ethically justified?']

In [ ]:
# huggingface access token
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Configuration
MODEL_ID = "NousResearch/Llama-2-7b-chat-hf"
MAX_NEW_TOKENS = 400                    # Adjust as needed

In [ ]:
device = torch.device("mps" if torch.has_mps else "cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.eval()

In [ ]:
# 2. Iterate and generate answers
answers = []
for question in tqdm(questions, desc="Generating answers"):
    inputs = tokenizer(question, return_tensors="pt").to(device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False
        )
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    answers.append(answer)

In [ ]:
# create csv of questions, answers
df = pd.DataFrame({'question': questions, 'answer': answers})
df.to_csv('shift-opinion-data.csv', index=False)

In [ ]:
answers